In [1]:
#!/usr/bin/env python
# # -*- coding: utf-8 -*-
#!/usr/bin/env python
#!pip install tensorflow==1.15

import numpy as np
import matplotlib.pyplot as plt
import pickle
import glob, os, inspect
import argparse
import time
import seaborn as sns
# import matplotlib
# matplotlib.use("Qt5Agg")
# #matplotlib.use('Agg')
# from mpl_toolkits.mplot3d import axes3d
# import matplotlib.pyplot as plt
import pandas as pd
import sys,inspect
sys.path.append("C:\\Users\\Vinamr J\\Documents\\qic\\qrc\\hqr\\hqr-master\\chaos\\Postprocess")
sys.path.append("C:\\Users\\Vinamr J\\Documents\\qic\\qrc\\hqr\\hqr-master\\chaos\\Methods\\Models\\Utils")

import utils  # for utils.py
from utils import *
import global_utils  # for global_utils.py

# import utils
# from utils import *
# import sys
# sys.path.append('../Methods/Models/Utils')
# import global_utils

In [2]:
sysname, tidx = "SST", 2
file_path = os.path.abspath("ESN_analysis.ipynb")
current_directory = os.path.dirname(file_path)
mid_directory = os.path.dirname(current_directory)
parent_directory = os.path.dirname(mid_directory)

print("Current Directory:", current_directory)
print("Parent Directory:", parent_directory)

#current_dir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
results_dir = os.path.join(os.path.dirname(mid_directory), "Results")  # Use os.path.join for path concatenation
print(results_dir)

eval_path = os.path.join(results_dir, sysname, 'Evaluation_Data')
print(eval_path)

model_path = os.path.join(results_dir, sysname, 'Trained_Models')
print(model_path)

fig_path = os.path.join(results_dir, sysname, 'Eval_Figures')
if not os.path.isdir(fig_path):
    os.makedirs(fig_path)  # Use os.makedirs to create parent directories if needed
print(fig_path)

Current Directory: c:\Users\Vinamr J\Documents\qic\qrc\hqr\hqr-master\chaos\Data\SST
Parent Directory: c:\Users\Vinamr J\Documents\qic\qrc\hqr\hqr-master\chaos
c:\Users\Vinamr J\Documents\qic\qrc\hqr\hqr-master\chaos\Results
c:\Users\Vinamr J\Documents\qic\qrc\hqr\hqr-master\chaos\Results\SST\Evaluation_Data
c:\Users\Vinamr J\Documents\qic\qrc\hqr\hqr-master\chaos\Results\SST\Trained_Models
c:\Users\Vinamr J\Documents\qic\qrc\hqr\hqr-master\chaos\Results\SST\Eval_Figures


In [6]:
models = []
samples = []
size_values = [40, 60, 80, 100, 120, 150, 200, 300, 500, 1000]
for size in size_values:
    string1 = f"ESN_pinv-RDIM_5-N_used_427-SIZE_{size}-D_10.0-RADIUS_0.9-SIGMA_1.0-DL_40-NL_0-IPL_300-REG_1e-07-NICS_3"
    string2 = f"ESN-{size}"
    models.append([os.path.join(eval_path,string1),string2])
    samples.append(string2)
title = "Sea Surface Temperatures"

In [3]:
# models = [[os.path.join(eval_path, "hqrc_pinv-RDIM_5-N_used_427-DL_40-Nqr_5-A_0.9-J_2.0-fJ_1-V_10-NL_0-IPL_20-IUL_0-REG_1e-07-AU_0-NICS_3"),"HQR-5,V-10,DL=40,IPL=20,n_tests=3"],
#           [os.path.join(eval_path, "hqrc_pinv-RDIM_5-N_used_427-DL_8-Nqr_5-A_0.9-J_2.0-fJ_1-V_10-NL_0-IPL_8-IUL_0-REG_1e-07-AU_0-NICS_1"),"HQR-5,V-10,DL=8,IPL=8,n_tests=1"],
#           [os.path.join(eval_path, "hqrc_pinv-RDIM_5-N_used_427-DL_40-Nqr_5-A_0.9-J_2.0-fJ_1-V_10-NL_0-IPL_300-IUL_0-REG_1e-07-AU_0-NICS_1"),"HQR-5,V-10,DL=40,IPL=300,n_tests=1"]]
# samples = ["HQR-5,V-10,DL=40,IPL=20,n_tests=3","HQR-5,V-10,DL=8,IPL=8,n_tests=1","HQR-5,V-10,DL=40,IPL=300,n_tests=1"]
# title = "Sea Surface Temperatures"

In [7]:
rmse_dict = dict()
targets = dict()
outputs = dict()
sp_outputs = dict()
sp_targets = dict()
Wouts = dict()
coeffs = dict()
rmnse_avg_train_dict = dict()
rmnse_avg_test_dict = dict()
for i in range(len(models)):
    rfolder, label = models[i][0], models[i][1]
    fname = os.path.join(rfolder, 'results.pickle')
    if os.path.isfile(fname):
        with open(fname, 'rb') as rfile:
            try:
                rs = pickle.load(rfile)
            except:
                continue
            #print(rs.keys())
            qs = QResults()
            qs.rmnse_avg_test = rs['rmnse_avg_TEST']
            qs.rmnse_avg_train = rs['rmnse_avg_TRAIN']
            qs.n_pred_005_avg_test = rs['num_accurate_pred_005_avg_TEST']
            qs.n_pred_005_avg_train = rs['num_accurate_pred_005_avg_TRAIN']
            qs.n_pred_050_avg_test = rs['num_accurate_pred_050_avg_TEST']
            qs.n_pred_050_avg_train = rs['num_accurate_pred_050_avg_TRAIN']
            qs.model_name = rs['model_name']
            #if qs.rmnse_avg_test != np.inf and qs.rmnse_avg_train != np.inf:
                #print(rs.keys())
            #print(qs.model_name)
            #print('train={}, test={}'.format(qs.rmnse_avg_train, qs.rmnse_avg_test))
            #qs.info()
            rmnse_avg_train_dict[label] = rs['rmnse_avg_TRAIN']
            rmnse_avg_test_dict[label] = rs['rmnse_avg_TEST']
            pred_test = rs['predictions_all_TEST']
            truth_test = rs['truths_all_TEST']
            print(pred_test.shape)
            M = len(pred_test)
            print('{} Number of test'.format(qs.model_name), M)
            rmsels = []
            for j in range(M):
                    rmsels.append(calNRMSE(pred_test[j], truth_test[j]))

            rmse_dict[label] = np.mean(np.array(rmsels), axis=0)
            print(rmse_dict[label].shape)


            targets[label] = truth_test[0]
            outputs[label] = pred_test[0]

            # For frequency
            sp_outputs[label] = rs['sp_pred_TEST']
            sp_targets[label] = rs['sp_true_TEST']
    else:
        print('Not found {}'.format(fname))
    # if trained_models is not None:
    #     rfolder, label = trained_models[i][0], trained_models[i][1]
    #     fname = os.path.join(rfolder, 'data.pickle')
    #     if os.path.isfile(fname):
    #         print('File existed: ', fname)
    #         with open(fname, 'rb') as rfile:
    #             #try:
    #             rs = pickle.load(rfile)
    #             #except:
    #             #    continue
    #             print(rs.keys())
    #             coeffs[label] = np.array(rs['coeffs'])
    #             Wouts[label]  = rs['W_out'][:-1].reshape((coeffs[label].shape[0], -1))
    #             print(Wouts[label].shape, coeffs[label].shape)
    #     else:
    #         print('Not found saved model {}'.format(fname))

(3, 300, 5)
ESN_pinv-RDIM_5-N_used_427-SIZE_40-D_10.0-RADIUS_0.9-SIGMA_1.0-DL_40-NL_0-IPL_300-REG_1e-07-NICS_3 Number of test 3
(300,)
(3, 300, 5)
ESN_pinv-RDIM_5-N_used_427-SIZE_60-D_10.0-RADIUS_0.9-SIGMA_1.0-DL_40-NL_0-IPL_300-REG_1e-07-NICS_3 Number of test 3
(300,)
(3, 300, 5)
ESN_pinv-RDIM_5-N_used_427-SIZE_80-D_10.0-RADIUS_0.9-SIGMA_1.0-DL_40-NL_0-IPL_300-REG_1e-07-NICS_3 Number of test 3
(300,)
(3, 300, 5)
ESN_pinv-RDIM_5-N_used_427-SIZE_100-D_10.0-RADIUS_0.9-SIGMA_1.0-DL_40-NL_0-IPL_300-REG_1e-07-NICS_3 Number of test 3
(300,)
(3, 300, 5)
ESN_pinv-RDIM_5-N_used_427-SIZE_120-D_10.0-RADIUS_0.9-SIGMA_1.0-DL_40-NL_0-IPL_300-REG_1e-07-NICS_3 Number of test 3
(300,)
(3, 300, 5)
ESN_pinv-RDIM_5-N_used_427-SIZE_150-D_10.0-RADIUS_0.9-SIGMA_1.0-DL_40-NL_0-IPL_300-REG_1e-07-NICS_3 Number of test 3
(300,)
(3, 300, 5)
ESN_pinv-RDIM_5-N_used_427-SIZE_200-D_10.0-RADIUS_0.9-SIGMA_1.0-DL_40-NL_0-IPL_300-REG_1e-07-NICS_3 Number of test 3
(300,)
(3, 300, 5)
ESN_pinv-RDIM_5-N_used_427-SIZE_300-D_1

In [19]:
model_test_error = []
test_avg_error = []
for label in samples:
    model_test_error.append([label,np.mean(rmse_dict[label])])
    test_avg_error.append(np.mean(rmse_dict[label]))
print(model_test_error)
print("Test Average error = ",np.mean(test_avg_error))


[['ESN-40', 1.6130774555297716], ['ESN-60', 1.307837479778843], ['ESN-80', 2.1239969867527946], ['ESN-100', 1.4380652503302187], ['ESN-120', 2.0849269689681544], ['ESN-150', 2.0469315105579873], ['ESN-200', 14.61995049569763], ['ESN-300', 53.64252693599409], ['ESN-500', 192.5343746203439], ['ESN-1000', 23.480702398181116]]
Test Average error =  29.489239010213446


In [18]:
print(rmnse_avg_train_dict)
print(rmnse_avg_test_dict)
error_array_test = []
error_array_train = []
for label in samples:
    error_array_train.append(rmnse_avg_train_dict[label])
    error_array_test.append(rmnse_avg_test_dict[label])
print("Train average error for ESN =", np.mean(error_array_train))
print("Test average error for ESN =", np.mean(error_array_test))

{'ESN-40': 1.021261638921574, 'ESN-60': 0.5289043299427124, 'ESN-80': 1.2769176622454805, 'ESN-100': 1.0806888105470531, 'ESN-120': 2.641591192422233, 'ESN-150': 1.1894740499679881, 'ESN-200': 11.292778038486684, 'ESN-300': 34.05851894216005, 'ESN-500': 63.96118907757753, 'ESN-1000': 17.61250607088196}
{'ESN-40': 1.453840151289397, 'ESN-60': 1.0833544858348954, 'ESN-80': 1.8841448664082783, 'ESN-100': 1.2806402020905332, 'ESN-120': 1.7847912553720777, 'ESN-150': 1.6978414255927141, 'ESN-200': 12.659072038990038, 'ESN-300': 40.75638342008179, 'ESN-500': 148.9765874832078, 'ESN-1000': 19.636961512611435}
Train average error for ESN = 13.466382981315325
Test average error for ESN = 23.121361684147892


In [20]:
models = []
samples = []
size_values = [40, 60, 80, 100, 120, 150, 200, 300, 500, 1000]
for size in size_values:
    string1 = f"ESN_pinv-RDIM_5-N_used_427-SIZE_{size}-D_10.0-RADIUS_0.9-SIGMA_1.0-DL_40-NL_0-IPL_300-REG_1e-07-NICS_3"
    string2 = f"ESN-{size}"
    models.append([os.path.join(model_path,string1),string2])
    samples.append(string2)
title = "Sea Surface Temperatures"

In [21]:
train_time_dict = dict()
memory_dict = dict()
trainable_parameters_dict = dict()
for i in range(len(models)):
    rfolder, label = models[i][0], models[i][1]
    fname = os.path.join(rfolder, 'data.pickle')
    if os.path.isfile(fname):
        with open(fname, 'rb') as rfile:
            try:
                rs = pickle.load(rfile)
            except:
                continue
            #print(rs.keys())
            train_time_dict[label]=rs["total_training_time"]
            trainable_parameters_dict[label]=rs["n_trainable_parameters"]
            memory_dict[label]=rs["memory"]
    else:
        print('Not found {}'.format(fname))

In [22]:
train_time_dict

{'ESN-40': 0.06892657279968262,
 'ESN-60': 0.12274408340454102,
 'ESN-80': 0.08296060562133789,
 'ESN-100': 0.08350563049316406,
 'ESN-120': 0.07953810691833496,
 'ESN-150': 0.09551262855529785,
 'ESN-200': 0.12108230590820312,
 'ESN-300': 0.1894676685333252,
 'ESN-500': 0.29119348526000977,
 'ESN-1000': 0.9104361534118652}

In [23]:
memory_dict

{'ESN-40': 96.953125,
 'ESN-60': 97.484375,
 'ESN-80': 98.16015625,
 'ESN-100': 98.37890625,
 'ESN-120': 98.734375,
 'ESN-150': 99.328125,
 'ESN-200': 100.21875,
 'ESN-300': 101.94921875,
 'ESN-500': 107.23828125,
 'ESN-1000': 128.93359375}

In [24]:
trainable_parameters_dict

{'ESN-40': 200,
 'ESN-60': 300,
 'ESN-80': 400,
 'ESN-100': 500,
 'ESN-120': 600,
 'ESN-150': 750,
 'ESN-200': 1000,
 'ESN-300': 1500,
 'ESN-500': 2500,
 'ESN-1000': 5000}